### Imports

In [1]:
from itertools import combinations
from typing import List, NoReturn

### Types

In [2]:
variable = int

list_of_variable = List[variable]
clauses = List[variable]
set_of_clauses = List[clauses]

In [31]:
def get_CNF_n_true_among(values: list_of_variable, n: int) -> set_of_clauses:
    """
    Returns the CNF equivalent to two true variables from the list provided in parameter

    Parameters
    ----------
    values : int
        List of logic variables
    n : int
        Number of true variables

    """
    if len(values) < n:
        raise ValueError("The size of the list must be at least 2.")

    close: set_of_clauses = []

    for j in range(len(values) + 1):
        if j != n:
            for cmb in combinations(values, j):
                close.append([-k if k in cmb else k for k in values])
    return close

def get_CNF_no_true_among_neighbor(var: variable, neighbor: list_of_variable) -> set_of_clauses:
    """
    Returns the CNF equivalent to no true variable among the neighbor of the variable provided in parameter

    Parameters
    ----------
    var : list_of_variable
        Initial variable
    neighbor : list_of_variable
        List of neighboor of the inital variable

    """
    ret: set_of_clauses = []

    for var_neighbor in neighbor:
        ret.append([-var, -var_neighbor])

    return ret

In [32]:
class LireGrille:
    def __init__(self, path: str):
        """
        Read the grid of Starbattle game file given in parameter
        """
        with open(path, "r") as f:  # open the file and read it
            self.data = f.read()

        self.grid: List[List[str]] = [line.split(" ") for line in self.data.strip().split("\n")]

        self.dim: int = len(self.grid[0])  # get the number of columns in the grid

    def get_line(self, line_number: int) -> list_of_variable:
        """
        Return all variables of the line given in parameter

        """
        valeur_deb = (line_number - 1) * self.dim + 1

        return list(range(valeur_deb, valeur_deb + self.dim))

    def get_column(self, col_number: int) -> list_of_variable:
        """
        Return all variables of the column given in parameter

        """
        return list(range(col_number, self.dim ** 2 + 1, self.dim))

    def get_zone(self, zone_number: int) -> list_of_variable:
        """
        Return all variables of the zone given in parameter
        """
        zone: list_of_variable = []
        k: int = 1
        for line in self.grid:
            for elt in line:
                if int(elt) == zone_number:
                    zone.append(k)
                k += 1
        return zone

    def get_neighbor(self, var: int) -> list_of_variable:
        """
        Return all variables of the neighbor of the variable given in parameter
        """

        l = [var + 1, var - 1, var - self.dim, var + self.dim, var - self.dim - 1,
             var - self.dim + 1, var + self.dim - 1, var + self.dim + 1]

        if var % self.dim == 1:  # if the variable is in the first column
            l.remove(var - 1)
            l.remove(var + self.dim - 1)
            l.remove(var - self.dim - 1)

        elif var % self.dim == 0:  # if the variable is in the last column
            l.remove(var + 1)
            l.remove(var + self.dim + 1)
            l.remove(var - self.dim + 1)

        if var <= self.dim:  # if the variable is in the first line
            if var % self.dim != 1:
                l.remove(var - self.dim - 1)
            if var % self.dim != 0:
                l.remove(var - self.dim + 1)
            l.remove(var - self.dim)

        elif var > self.dim * (self.dim - 1):  # if the variable is in the last line
            if var % self.dim != 1:
                l.remove(var + self.dim - 1)
            if var % self.dim != 0:
                l.remove(var + self.dim + 1)
            l.remove(var + self.dim)

        return l

    def get_dimension(self) -> int:
        """
        Return the number of lines/column/zone in the grid
        """
        return self.dim


In [33]:
def get_battle_star_clauses(grid: LireGrille, n: int) -> set_of_clauses:
    """
    Returns the CNF equivalent to the Starbattle game

    Parameters
    ----------
    grid : Lire_grille
        Grid of the Starbattle game
    n : int
        Number of star by line/column/zone

    """
    ret: set_of_clauses = []

    for i in range(1, grid.get_dimension() + 1):
        ret.extend(get_CNF_n_true_among(grid.get_zone(i), n))
        ret.extend(get_CNF_n_true_among(grid.get_line(i), n))
        ret.extend(get_CNF_n_true_among(grid.get_column(i), n))

    for i in range(1, grid.get_dimension() ** 2 + 1):
        ret.extend(get_CNF_no_true_among_neighbor(i, grid.get_neighbor(i)))

    return ret

def write_DIMAC_clause(path: str, set: set_of_clauses, nb_of_variable: int) -> NoReturn:
    """
    Write the clause in DIMAC format in the file given in parameter

    Parameters
    ----------
    path : str
        Path of the file where the clause will be written
    set : set_of_clauses
        Clauses to write in DIMAC format
    nb_of_variable : int
        Number of variable of the grid
    """


    with open(path, 'w') as f:
        f.write(f"p cnf {nb_of_variable} {len(set)}\n")
        for clause in set:
            for var in clause:
                f.write(f"{var} ")
            f.write("0\n")

def run_clauses_generator(path: str, n: int) -> NoReturn:
    """
    Generate the clauses of the Starbattle game and write them in DIMAC format in the file given in parameter

    Parameters
    ----------
    path : str
        Path of the file where the clause will be written
    n : int
        Number of star by line/column/zone
    """
    grid: LireGrille = LireGrille(path)
    clauses: set_of_clauses = get_battle_star_clauses(grid, n)
    write_DIMAC_clause(path.split(".")[-2]+".dimac", clauses, grid.get_dimension() ** 2)

In [34]:
run_clauses_generator("Exemples/test3.txt", 1)